In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [2]:
df_true = pd.read_csv("data/True.csv")
df_fake = pd.read_csv("data/Fake.csv")

In [3]:
df_true.shape, df_fake.shape

((21417, 4), (23481, 4))

In [4]:
df_true['class'] = 1
df_fake['class'] = 0

In [5]:
df_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
df_fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [7]:
df = pd.concat([df_fake, df_true], axis = 0)
df.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
df = df.sample(frac = 1)
df.head()

,title,text,subject,date,class
955,MSNBC Producer Reminds Idiot Trump That ‘Morn...,Donald Trump spent his week on Twitter blastin...,News,"July 1, 2017",0
5452,Bill Maher Lets Us Know The ’25 Things’ We Ha...,Almost every well-known celebrity will likely ...,News,"July 16, 2016",0
13492,"Passenger train derails in Spain, 21 hurt",MADRID (Reuters) - A passenger train derailed ...,worldnews,"November 29, 2017",1
3175,Pence hires counsel to deal with Russia probe ...,WASHINGTON (Reuters) - U.S. Vice President Mik...,politicsNews,"June 15, 2017",1
19014,Brexit blues? Britons stay cheerful in tumultu...,LONDON (Reuters) - Happiness levels in Britain...,worldnews,"September 26, 2017",1


In [9]:
df.drop(columns = ['title', 'subject', 'date'], inplace = True)
df.head()

,text,class
955,Donald Trump spent his week on Twitter blastin...,0
5452,Almost every well-known celebrity will likely ...,0
13492,MADRID (Reuters) - A passenger train derailed ...,1
3175,WASHINGTON (Reuters) - U.S. Vice President Mik...,1
19014,LONDON (Reuters) - Happiness levels in Britain...,1


In [10]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [11]:
df.head()

,text,class
0,Donald Trump spent his week on Twitter blastin...,0
1,Almost every well-known celebrity will likely ...,0
2,MADRID (Reuters) - A passenger train derailed ...,1
3,WASHINGTON (Reuters) - U.S. Vice President Mik...,1
4,LONDON (Reuters) - Happiness levels in Britain...,1


In [12]:
def reg_exp(df):
    df = df.lower()
    df = re.sub('\n', '', df)
    df = re.sub('\[.*?\]', '', df)
    df = re.sub("\\W"," ",df) 
    df = re.sub('https?://\S+|www\.\S+', '', df)
    df = re.sub('<.*?>+', '', df)
    df = re.sub('\w*\d\w*', '', df)
    df = re.sub('[%s]' % re.escape(string.punctuation), '', df)
    return df

In [13]:
df.text = df["text"].apply(reg_exp)

In [14]:
X = df["text"]
y = df["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [15]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

LogisticRegression()

In [17]:
y_pred=log_reg.predict(X_test)
accuracy_score(y_test, y_pred)

0.985478841870824

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5887
           1       0.98      0.99      0.98      5338

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



Decision Tree

In [19]:
from sklearn.tree import DecisionTreeClassifier

des_tree = DecisionTreeClassifier()
des_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [20]:
y_pred = des_tree.predict(X_test)
accuracy_score(y_test, y_pred)

0.9948329621380846

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5887
           1       0.99      1.00      0.99      5338

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier

rand_forest = RandomForestClassifier(random_state=0)
rand_forest.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [23]:
y_pred = rand_forest.predict(X_test)
accuracy_score(y_test, y_pred)

0.9889532293986637

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5887
           1       0.99      0.99      0.99      5338

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

